<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/%E4%BA%88%E6%B8%AC%E5%88%B6%E5%BE%A1%EF%BC%88MPC%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: モデル予測制御（MPC）の例題を見せてください。plotlyを使用してください。

import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

# システムモデル (例: 1次遅れ系)
def system_model(x, u):
  A = 0.9
  B = 0.1
  x_next = A * x + B * u
  return x_next

# コスト関数 (例: 状態と入力の2乗和)
def cost_function(x_pred, u_pred, x_ref, u_ref):
  Q = 1
  R = 0.1
  cost = np.sum((x_pred - x_ref)**2 * Q) + np.sum((u_pred - u_ref)**2 * R)
  return cost

# MPC制御器
def mpc_controller(x_current, x_ref, prediction_horizon):
  # 制御入力の初期値
  u_init = np.zeros(prediction_horizon)

  # 最適化問題を解く
  def objective_function(u_pred):
    x_pred = np.zeros(prediction_horizon + 1)
    x_pred[0] = x_current
    for i in range(prediction_horizon):
      x_pred[i+1] = system_model(x_pred[i], u_pred[i])
    return cost_function(x_pred[1:], u_pred, x_ref, 0)

  # 制約条件 (例: 入力の上限と下限)
  bounds = [(-1, 1)] * prediction_horizon

  result = minimize(objective_function, u_init, bounds=bounds)
  u_optimal = result.x

  return u_optimal[0]

# シミュレーションパラメータ
prediction_horizon = 5
simulation_steps = 20
x_ref = 1.0

# 初期状態
x_current = 0.0

# シミュレーション
x_history = [x_current]
u_history = []
for _ in range(simulation_steps):
  u_current = mpc_controller(x_current, x_ref, prediction_horizon)
  x_next = system_model(x_current, u_current)
  x_history.append(x_next)
  u_history.append(u_current)
  x_current = x_next

# 結果のプロット
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(simulation_steps + 1)), y=x_history, mode='lines', name='状態'))
fig.add_trace(go.Scatter(x=list(range(simulation_steps)), y=u_history, mode='lines', name='入力'))
fig.add_trace(go.Scatter(x=list(range(simulation_steps + 1)), y=[x_ref] * (simulation_steps + 1), mode='lines', name='目標値'))
fig.update_layout(title='MPC制御シミュレーション', xaxis_title='時間ステップ', yaxis_title='状態/入力')
fig.show()